## Actividad 8 - Ejercicio 4

El problema de la mochila también puede resolverse utilizando la programación genética. Identifica una problemática relacionada con los fundamentos de este ejemplo y desarrolla un programa en Python donde implementes un algoritmo genético para resolverlo. Repite el proceso, pero utilizando la programación dinámica y realiza una comparación sobre los resultados obtenidos.

Se presenta a continuación el desarrollo de este ejercicio.

**Problema de obtener el cambio mínimo de monedas**
Este problema trata de obtener el cambio de un valor dado en centavos teniendo cantidad infinita de centavos de cada denominación $$C={c1, c2,..., cm}$$.
Dado un arreglo de denominaciones disponibles $$monedas$$Y un valor de cambio $$cambio$$Se desea obtener el cambio mínimo de monedas que se puede obtener con el valor de cambio dado.

**Se presenta el desarrollo de este ejercicio usando programación dinámica.**

In [6]:
import sys # Esta librería nos permite obtener el tamaño maximo del arreglo

In [7]:
def minCoins(monedas, cambio, it):
    n = len(monedas)
    # define dp array
    dp  = [[None] * (cambio + 1) for _ in range(n + 1)]

    # Inicializando el arreglo
    #Si el precio es o las monedas requridas son 0, entonces no hay monedas necesarias y dp[i][0] = 0
    #Si el numero de monedas es 0 entonces se necesitan monedas infinitas para pagar el precio y dp[0][j] = inf-1 para evitar overflow

    for j in range(cambio + 1):
        dp[0][j] = sys.maxsize -1
    for i in range(n+1):
        dp[i][0] = 0

    #Una moneda solo puede elegirse si su valor es menor que el precio requerido
    for i in range(1,n+1):
        for j in range(1, cambio + 1):
            if monedas[i - 1] <= j:
                dp[i][j] = min(dp[i - 1][j], 1 + dp[i][j-monedas[i-1]]);
            else:
                dp[i][j] = dp[i - 1][j];

    total = dp[n][cambio]
    print(f"El total de valores para el cambio minimo es: {total}")


In [8]:
monedas = [1,4,6,8]
cambio = 12
items = len(monedas)
minCoins(monedas,cambio, items)

El total de valores para el cambio minimo es: 2


**Solución usando algoritmo genetico**
Se presenta a continuación una solución a este problema que utiliza un algoritmo genético para la misma.

In [9]:
import random
import pandas as pd

import warnings

warnings.filterwarnings('ignore')

d = [1, 4, 6, 8, 'Fitness']  # denominaciones de monedas
N = 12  # monto total
size = 10  # tamaño de la población

# Generar una población inicial de 10 individuos
poblacion = []
for i in range(size):
    c = []
    for j in range(len(d)):
        c.append(random.randint(0, N))
    poblacion.append(c)

poblacion = pd.DataFrame(poblacion, columns=d)
poblacion = poblacion.astype({'Fitness': float})
for i in range(size):
    fitness = 0
    for j in range(len(d) - 1):
        fitness += poblacion[d[j]][i] * d[j]

    fitness = abs(fitness - N)
    poblacion['Fitness'][i] = 1 / (1 + float(fitness))

pop = poblacion.copy()
pop['Fitness'] = round(pop['Fitness'], 4)
print(pop)

# algoritmo genético
cnt = 1
while (True):
    for i in range(size):
        fitness = 0
        for j in range(len(d) - 1):
            fitness += poblacion[d[j]][i] * d[j]

        fitness = abs(fitness - N)
        poblacion['Fitness'][i] = 1 / (1 + float(fitness))

    poblacion = poblacion.sort_values(by=['Fitness'])

    parents = poblacion[-6:].reset_index()
    offspring = poblacion[:3].reset_index()

    # crossover
    for i in range(3):
        r = random.randint(0, len(d) - 1)
        for j in range(len(d) - 1):
            if (j < r):
                offspring[d[j]][i] = parents[d[j]][i]
            else:
                offspring[d[j]][i] = parents[d[j]][6 - i - 1]

    # mutación
    mutacion_p = 0.75  # probabilidad de mutación
    for i in range(3):
        for j in range(len(d) - 1):
            p = random.random()
            if (p > mutacion_p):
                offspring[d[j]][i] = random.randint(0, N)
    poblacion = poblacion[3:]
    poblacion = poblacion.append(offspring, ignore_index=True)
    poblacion = poblacion.drop(['index'], axis=1)

    pop = poblacion.copy().sort_values(by=['Fitness']).reset_index()
    pop['Fitness'] = round(pop['Fitness'], 4)
    pop = pop.drop(['index'], axis=1)

    if (max(poblacion['Fitness']) == 1):
        break

    if (cnt % 100 == 0):
        print(max(poblacion['Fitness']))
    cnt += 1

poblacion = poblacion.sort_values(by=['Fitness']).reset_index()
poblacion = poblacion[-1:]
print(poblacion)
totalg = (poblacion == 0).sum(axis=1)
print(f"The total de valores para el cambio minimo es: {((len(d)-1)-totalg).to_string(index = False, header = False)}")

    1   4   6   8  Fitness
0   4   9  11  12   0.0052
1   3   1   4   0   0.0500
2   3   5   8   8   0.0081
3   9   8   4   3   0.0128
4  11   0   8  11   0.0074
5   2   5   2   5   0.0159
6  10   0  10   3   0.0120
7  11   2   0   5   0.0208
8   1  12   0  10   0.0085
9   2   9   2   0   0.0256
   index  1  4  6  8  Fitness
9      6  8  1  0  0      1.0
The total de valores para el cambio minimo es: 2
